In [1]:
import pandas as pd
import numpy as np

In [2]:
!gdown 1N87vmtobGYPZosrd_dE7RHw-c0RYXeZG

Downloading...
From: https://drive.google.com/uc?id=1N87vmtobGYPZosrd_dE7RHw-c0RYXeZG
To: /content/sarcasmania-dataset.csv
100% 4.17M/4.17M [00:00<00:00, 245MB/s]


In [3]:
data = pd.read_csv('sarcasmania-dataset.csv')

In [4]:
data['tweet'] = data['tweet'].str.replace(r'[@]+[A-Za-z0-9_-]+', '', regex=True)

In [5]:
data = data.rename(columns={'tweet': 'comment'})

In [6]:
ann_columns = ['humor']

In [7]:
data

,id,comment,sarcasm,humor,insult
0,1,i hope youre lurking rn. i want to listen to ...,0,0,0
1,2,05 really taught me a valuable lesson I'm neve...,0,1,1
2,3,"Never had a voice to protest, so you fed me s...",0,0,0
3,4,Rest in peace & love to you and your family,0,0,1
4,5,100 days until Christmas! 🌲 #too soon #not rea...,0,0,1
...,...,...,...,...,...
39775,39776,I could see the makeup artists giving u an As...,1,0,1
39776,39777,Slvr... That's great name #NOT 😂 😂 😂 #sarcasm,1,1,0
39777,39778,He is the fag we need but not the fag we des...,1,1,1
39778,39779,Zuma sounding like Kanye West right now trying...,1,0,1


In [9]:
# test set (preso dall'excel)
test_set = pd.read_csv('05_test_set.csv')

print(test_set)

                                                  text annotation
0                                 Can you please reply  not funny
1    and the clip of this song is making me cry. ju...      funny
2      5 papers to write by Tuesday #challengeaccepted  not funny
3    A man that is showing his wedding vows do matt...  not funny
4    I am so happy to see the humble side of #Trump...      funny
..                                                 ...        ...
995  Yes.. please stalk me so I know you're an inse...  not funny
996  Yes thanks piss me off even more than I alread...      funny
997  You'd make a really cool ghost! " is a new pos...      funny
998  You know you need to get a life when you even ...      funny
999  You're such a burnt out zombie, you always suc...      funny

[1000 rows x 2 columns]


In [10]:
import random

# 5 frasi random not funny che non sono in test
random_sentences_0 = data[data['humor'] == 0]['comment'].sample(10).reset_index(drop=True)
random_sentences_0 = random_sentences_0[~random_sentences_0.isin(test_set['text'])].reset_index(drop=True)

# 5 frasi random funny che non sono in test
random_sentences_1 = data[data['humor'] == 1]['comment'].sample(10).reset_index(drop=True)
random_sentences_1 = random_sentences_1[~random_sentences_1.isin(test_set['text'])].reset_index(drop=True)

sentences_group_0 = list(random_sentences_0)
sentences_group_1 = list(random_sentences_1)


In [11]:
# nuove colonne per i prompt
for n in [10, 8, 6, 4, 2]:
    test_set[f'prompt{n}'] = (
        'Which one of the attributes: "funny", "not funny" describes a given text?\n'
        ' Write your answer in the form of a Python list containing the appropriate attribute.\n\n'
        'These are examples of "funny" sentences:\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_1[:n]) +
        '\n\nThese are examples of "not funny" sentences:\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_0[:n]) +
        "\n\nText: " + test_set['text']
    )

print(test_set.head(10))

                                                text annotation  \
0                               Can you please reply  not funny   
1  and the clip of this song is making me cry. ju...      funny   
2    5 papers to write by Tuesday #challengeaccepted  not funny   
3  A man that is showing his wedding vows do matt...  not funny   
4  I am so happy to see the humble side of #Trump...      funny   
5  Accept the things to which fate binds you, and...  not funny   
6           I don't give a fuck, I don't give a fuck  not funny   
7  After a hugely successful year and meeting so ...      funny   
8  After four years and a lot of money spent I've...      funny   
9  After the second week she be stuffin the turke...      funny   

                                            prompt10  \
0  Which one of the attributes: "funny", "not fun...   
1  Which one of the attributes: "funny", "not fun...   
2  Which one of the attributes: "funny", "not fun...   
3  Which one of the attributes: "funny

In [12]:
# test su csv
test_set.head(10).to_csv("output_file.csv", index=False)

In [13]:
# colonne per csv totale
selected_columns = ["text", "annotation", "prompt10", "prompt8", "prompt6", "prompt4", "prompt2"]

# Salva il DataFrame con solo le colonne selezionate in un file CSV
test_set[selected_columns].to_csv("05_Sarcasm2.csv", index=False)